# Prueba técnica ecosistemas

In [5]:
# Importamos las librerias requeridas para la implementación del ejercicio

import sqlite3
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

Implementamos la clase Calculo_comisiones para definir las funciones que necesitamos para
realizar el ejercicio propuesto, entre estas, se encuentra la función obtener_datos, para extraer la información de la base datos en SQlite proporcionada. Tambien, definimos la función cobro_peticiones donde se automatizan las condiciones de cobro requeridas para cada una de las empresas, considerando la cantidad total de peticiones, las que fueron exitosas, las peticiones que no fueron exitosas, así como los cobros corresponientes incluyendo tanto el IVA como los descuentos aplicables. 

In [2]:
class Calculo_comisiones:
    
    def __init__(self, conn):
        self.conn = conn
        self.cursor = self.conn.cursor()

    # Consulta SQL para obtener los datos requeridos cruzando ambas tablas
    def obtener_datos(self, mes):
        query = f"""
        CREATE TABLE IF NOT EXISTS resultado AS
        SELECT A.*, C.*
        FROM apicall AS A
        LEFT JOIN commerce AS C
        ON A.commerce_id = C.commerce_id
        WHERE strftime('%Y-%m', A.date_api_call) = '{mes}'
        """
        self.cursor.execute(query)
        self.conn.commit()

    def cobro_peticiones(self, empresa, peticiones_exitosas, peticiones_fallidas, peticiones_totales, precios, iva):
        total = 0
        if empresa == 'Innovexa Solutions':
            total = peticiones_exitosas * precios['Innovexa Solutions'] + peticiones_exitosas * precios['Innovexa Solutions'] * iva
        elif empresa == 'NexaTech Industries':
            if peticiones_totales <= 10000:
                total = peticiones_exitosas*precios['NexaTech Industries'][0] + peticiones_exitosas*precios['NexaTech Industries'][0]*iva
            elif peticiones_totales <= 20000:
                total = peticiones_exitosas*precios['NexaTech Industries'][1] + peticiones_exitosas*precios['NexaTech Industries'][1]*iva
            else:
                total = peticiones_exitosas*precios['NexaTech Industries'][2] + peticiones_exitosas*precios['NexaTech Industries'][2]*iva
        elif empresa == 'QuantumLeap Inc.':
            total = peticiones_exitosas*precios['QuantumLeap Inc.'] + peticiones_exitosas*precios['QuantumLeap Inc.']*iva
        elif empresa == 'Zenith Corp.':
            if peticiones_totales >= 0 and peticiones_totales <= 22000:
                total = peticiones_exitosas*precios['Zenith Corp.'][0] + peticiones_exitosas*precios['Zenith Corp.'][0]*iva
            elif peticiones_totales >= 22001:
                total = peticiones_exitosas*precios['Zenith Corp.'][1] + peticiones_exitosas*precios['Zenith Corp.'][1]*iva
            if peticiones_fallidas >= 6000:
                total = total - total*0.05
        elif empresa == 'FusionWave Enterprises':
            total = peticiones_exitosas*precios['FusionWave Enterprises'] + peticiones_exitosas*precios['FusionWave Enterprises']*iva
            if 2500 <= peticiones_fallidas <= 4500:
                total = total - total*0.05  # Descuento del 5% antes de IVA
            elif peticiones_fallidas > 4501:
                total = total - total*0.08
        return total

    # Función principal con parametrización de mes, precios e IVA
    def calcular_comision(self, mes, precios, iva):
        # Ejecutar la consulta SQL y crear la tabla temporal 'resultado'
        self.obtener_datos(mes)
        
        # Leer los datos de la tabla resultado desde SQLite a un DataFrame de pandas
        resultado = pd.read_sql("SELECT * FROM resultado", self.conn)
        commerce = pd.read_sql("SELECT * FROM commerce", self.conn)
        
        # Agrupar por 'commerce_id' y 'ask_status' para contar peticiones exitosas y fallidas
        resultado_tab = pd.crosstab(resultado['commerce_id'], resultado['ask_status'])
        resultado_tab['Total'] = resultado_tab['Successful'] + resultado_tab['Unsuccessful']

        # Unir 'resultado_tab' con la tabla 'commerce' para obtener nombre, NIT y correo
        resultado_tab = resultado_tab.merge(commerce[['commerce_id', 'commerce_name', 'commerce_nit', 'commerce_email']], 
                                            left_on='commerce_id', right_on='commerce_id', how='left')
        
        # Aplicar la función de cobro a cada empresa
        resultado_tab['Cobro Total'] = resultado_tab.apply(
            lambda row: self.cobro_peticiones(row['commerce_name'], row['Successful'], row['Unsuccessful'], row['Total'], precios, iva), axis=1)

        # Añadir los campos requeridos directamente a resultado_tab
        resultado_tab['Fecha-Mes'] = mes  # Mes parametrizado
        resultado_tab['Valor_comision'] = resultado_tab['Cobro Total'] / (1 + iva)  # Comisión sin IVA
        resultado_tab['Valor_iva'] = resultado_tab['Valor_comision'] * iva  # IVA
        resultado_tab['Valor_Total'] = resultado_tab['Cobro Total']  # Total con IVA
        
        return resultado_tab

In [3]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('C:/Prueba_ecosistemas/database.sqlite')

# Crear instancia de la clase
calculo = Calculo_comisiones(conn)

NameError: name 'sqlite3' is not defined

In [ ]:
# Definir precios y el IVA
precios = {
    'Innovexa Solutions': 300,
    'NexaTech Industries': [250, 200, 170],
    'QuantumLeap Inc.': 600,
    'Zenith Corp.': [250, 130],
    'FusionWave Enterprises': 300
}
iva = 0.19
mes = '2024-07'  

In [ ]:
resultado_tab = calculo.calcular_comision(mes, precios, iva)

# Filtrar columnas relevantes
df_final = resultado_tab[['Fecha-Mes', 'commerce_name', 'commerce_nit', 'Valor_comision', 'Valor_iva', 'Valor_Total', 'commerce_email']]
df_final = df_final.rename(columns={'commerce_name': 'Nombre', 'commerce_nit': 'Nit', 'commerce_email': 'Correo'})

